In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path('../..')
ROOT.resolve()

In [ ]:
# Read file, sheet, headder
dwelling_stock = pd.read_excel(ROOT / 'raw/vacant-homes/LT_125.ods', engine='odf', sheet_name='LT_125_unrounded', header=5)

# Drop unused column
dwelling_stock.drop(columns=["Notes"], inplace=True)

# Regex pattern (case-insensitive removal of brackets and 'Notes')
pattern = r'(?i)\[.*?\]|\bnotes\b'

# Clean column names
dwelling_stock.columns = dwelling_stock.columns.str.replace(pattern, '', regex=True).str.strip()

# Make a list of dates
dates = dwelling_stock.columns.to_list()[2:]

# Pivot the dataframe
dwelling_stock = dwelling_stock.melt(id_vars=['New ONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)

# Add measure column
dwelling_stock['Measure'] = 'Dwellings'

# Rename columns
dwelling_stock.rename(columns={'New ONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)

# Replace values
dwelling_stock.replace({'NORTH EAST': 'North East', 
                        'NORTH WEST': 'North West', 
                        'YORKSHIRE AND THE HUMBER': 'Yorkshire and The Humber', 
                        'EAST MIDLANDS': 'East Midlands', 
                        'WEST MIDLANDS': 'West Midlands', 
                        'EAST OF ENGLAND': 'East of England', 
                        'LONDON': 'London', 
                        'SOUTH EAST': 'South East',
                        'SOUTH WEST': 'South West',
                        'ENGLAND': 'England',
                        'E11000004': 'E11000007',
                        'St. Albans': 'St Albans',
                        'St Helens': 'St. Helens',
                        'Folkestone and Hythe [note 7]': 'Folkestone and Hythe',
                        'St. Edmundsbury': 'St Edmundsbury',
                        r"&": 'and'}, inplace=True)

# Limit dates to after 2004.
dwelling_stock = dwelling_stock.loc[dwelling_stock['date'].astype(int) >= 2004]

# View table
dwelling_stock

In [ ]:
long_term_vacants = pd.read_excel(ROOT / 'raw/vacant-homes/LT_615.ods', engine='odf', sheet_name='All_long_term_vacants', header=2)
mapper = {}
for col in long_term_vacants.columns.to_list()[2:]:
    mapper[col] = col[6:10]
long_term_vacants.rename(columns=mapper, inplace=True)
dates = long_term_vacants.columns.to_list()[2:]
long_term_vacants = long_term_vacants.melt(id_vars=['ONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)
long_term_vacants['Measure'] = 'LongTermVacants'
long_term_vacants.rename(columns={'ONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)
long_term_vacants['geography_name'] = long_term_vacants['geography_name'].str.replace(r"&", 'and', regex=True)
long_term_vacants

In [ ]:
all_vacants = pd.read_excel(ROOT / 'raw/vacant-homes/LT_615.ods', engine='odf', sheet_name='All_vacants', header=2)
mapper = {}
for col in all_vacants.columns.to_list()[2:]:
    mapper[col] = col[6:10]
all_vacants.rename(columns=mapper, inplace=True)
dates = all_vacants.columns.to_list()[2:]
all_vacants = all_vacants.melt(id_vars=['ONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)
all_vacants['Measure'] = 'AllVacants'
all_vacants.rename(columns={'ONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)
all_vacants['geography_name'] = all_vacants['geography_name'].str.replace(r"&", 'and', regex=True)
all_vacants

In [ ]:
projections = pd.read_excel(ROOT / 'raw/household-projection/2018basedhhpsprincipalprojection.xlsx', sheet_name='406', header=4, engine='openpyxl')
projections = projections.melt(id_vars=['Area code', 'Area name'], var_name='date', value_vars=projections.columns.to_list()[1:], value_name='Value')
projections['Measure'] = 'HouseholdProjection'
projections.rename(columns={'Area code': 'geography_code', 'Area name': 'geography_name'}, inplace=True)
projections = projections.loc[(projections['date'].astype(int) <= 2023) & (projections['date'].astype(int) >= 2004)]
projections

In [ ]:
def fill_new_la_codes(data, old_codes, new_code, new_name):
    '''Use the old LADs to backfill data for new LADs'''
    # Find the old codes from the data and get a copy
    old_codes_data = data[data.geography_code.isin(old_codes)].copy()
    # Replace any [x] missing values with NaN
    old_codes_data.replace('[x]', np.nan, inplace=True)
    # Drop the NA values.
    old_codes_data.dropna(inplace=True)
    # Group the old codes by date and measure, and sum the 'value' column, then reset the index.
    new_code_data = old_codes_data.groupby(['date', 'Measure'])['Value'].sum().reset_index()
    # Set the name and code for the new data.
    new_code_data[['geography_code', 'geography_name']] = new_code, new_name
    # Drop the old codes from the data with a negation (~)
    data = data[~data.geography_code.isin(old_codes)]
    # Add the new data to the dataframe.
    result = pd.concat([data, new_code_data])

    return result


In [ ]:
combined = pd.concat([dwelling_stock, long_term_vacants, projections, all_vacants])

old_codes = [
    ['E07000026', 'E07000028', 'E07000029'], 
    ['E07000163', 'E07000164', 'E07000165', 'E07000166', 'E07000167', 'E07000168', 'E07000169'], 
    ['E07000187', 'E07000188', 'E07000246', 'E07000189'],
    ['E07000027', 'E07000030', 'E07000031']]
new_codes = ['E06000063', 'E066000065', 'E06000066', 'E06000064']
new_names = ['Cumberland', 'North Yorkshire', 'Somerset', 'Westmorland and Furness']

for i, j, k in zip(old_codes, new_codes, new_names):
    combined = fill_new_la_codes(combined, i, j, k)

In [ ]:
combined_long = combined.sort_values(by=['geography_code', 'Measure', 'date'])
combined_long = combined_long[['geography_code','geography_name','Measure','date','Value']]
combined_long['date'] = combined_long['date'].astype(int)
combined_long['Value'] = combined_long['Value'].astype(float, errors='ignore').round(0)
combined_long.replace('[x]', pd.NA, inplace=True)
combined_long.dropna(inplace=True)
combined_long['Value'] = combined_long['Value'].round(0).astype(int)

combined_long['geography_name'] = combined_long['geography_name'].str.replace('UA', '')
combined_long['geography_name'] = combined_long['geography_name'].str.replace(' (Met County)', '')
combined_long['geography_name'] = combined_long['geography_name'].str.strip()

combined_long.to_csv(ROOT / 'data/vacant-homes/standard/AllCombined_Cleaned_2024.csv', index=False)

In [ ]:
combined_wide = combined_long.pivot(index=["geography_code", "geography_name", "date"], columns="Measure", values="Value")

combined_wide.to_parquet(ROOT / 'data/vacant-homes/site/absolute.parquet')

In [ ]:
combined_wide['LongTermVacants_pct'] = (combined_wide['LongTermVacants'] / combined_wide['Dwellings']).mul(100).round(2)
combined_wide['AllVacants_pct'] = (combined_wide['AllVacants'] / combined_wide['Dwellings']).mul(100).round(2)
percentages_wide = combined_wide[['LongTermVacants_pct', 'AllVacants_pct']].dropna()
percentages_wide.to_parquet(ROOT / 'data/vacant-homes/site/percentages.parquet')

In [ ]:
# percentages_wide.to_csv(ROOT / 'data/vacant-homes/percentages.csv')


In [ ]:
most_recent = percentages_wide[percentages_wide.index.get_level_values('date') == 2023]

In [ ]:
ltv = most_recent.copy()
for i, row in most_recent.iterrows():
    ltv.loc[i, 'title'] = 'Long term vacants'
    ltv.loc[i, 'value'] = row['LongTermVacants_pct']
    eng_vals = most_recent[most_recent.index.get_level_values('geography_name')=='England']
    ltv.loc[i, 'footnote'] = f"of homes are long term vacant compared to {eng_vals['LongTermVacants_pct'].values[0]}% for all England"

av = ltv.copy()
for i, row in av.iterrows():
    av.loc[i, 'title'] = 'All vacants'
    av.loc[i, 'value'] = row['AllVacants_pct']
    eng_vals = most_recent[most_recent.index.get_level_values('geography_name')=='England']
    av.loc[i, 'footnote'] = f"of homes are vacant compared to {eng_vals['AllVacants_pct'].values[0]}% for all England"

av.drop(columns=['LongTermVacants_pct', 'AllVacants_pct'], inplace=True)
ltv.drop(columns=['LongTermVacants_pct', 'AllVacants_pct'], inplace=True)
ltv.index = ltv.index.droplevel([1, 2])
av.index = av.index.droplevel([1, 2])
headlines = pd.concat([ltv, av])

In [ ]:
headlines.to_parquet(ROOT / 'data/vacant-homes/site/headlines.parquet')
headlines.to_csv(ROOT / 'data/vacant-homes/standard/headlines.csv')